In [ ]:
import pandas as pd
import polars as pl
import threading
import panel as pn
from panel.reactive import ReactiveHTML
import param

from shapely import Polygon

import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

#
# ReactiveHTML Class for Panel Implementation
#
class RTGraphInteractHTML(ReactiveHTML):
    #
    # Inner Modification for RT SVG Render
    #
    # Initial Picture Is A Computer Mouse:  Source & License:
    #
    # https://www.svgrepo.com/svg/24318/computer-mouse
    #
    # https://www.svgrepo.com/page/licensing/#CC0
    #
    mod_inner = param.String(default="""
<svg fill="#000000" version="1.1" id="Capa_1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" 
	width="800px" height="800px" viewBox="0 0 800 800" xml:space="preserve" >
  <rect x="0" y="0" width="800" height="800" fill="#ffffff"/> <g> <g>
		<path d="M25.555,11.909c-1.216,0-2.207,1.963-2.207,4.396c0,2.423,0.991,4.395,2.207,4.395c1.208,0,2.197-1.972,2.197-4.395
			C27.751,13.872,26.762,11.909,25.555,11.909z"/>
		<path d="M18.22,5.842c4.432,0,6.227,0.335,6.227,3.653h2.207c0-5.851-4.875-5.851-8.433-5.851c-4.422,0-6.227-0.326-6.227-3.644
			H9.795C9.795,5.842,14.671,5.842,18.22,5.842z"/>
		<path d="M29.62,9.495c0.209,0.632,0.331,1.315,0.331,2.031v9.548c0,2.681-1.562,4.91-3.608,5.387
			c0.004,0.031,0.021,0.059,0.021,0.1v7.67c0,0.445-0.363,0.81-0.817,0.81c-0.445,0-0.809-0.365-0.809-0.81v-7.67
			c0-0.041,0.019-0.068,0.022-0.1c-2.046-0.477-3.609-2.706-3.609-5.387v-9.548c0-0.715,0.121-1.399,0.331-2.031
			c-6.057,1.596-10.586,7.089-10.586,13.632v12.716c-0.001,7.787,6.37,14.158,14.155,14.158h0.999
			c7.786,0,14.156-6.371,14.156-14.158V23.127C40.206,16.584,35.676,11.091,29.62,9.495z"/>
	</g> </g> </svg>
    """)

    #
    # Selection Path
    #
    selectionpath = param.String(default="M 0 0 L 10 10 L 0 10 Z")

    #
    # Panel Template
    # - Just copy & paste the template in __init__ and fill in the variables w/ 128 x 128
    #
    _template = f'<svg tabindex="1" id="myparent" width="128" height="128" '                    + \
                """ onkeypress="${script('keyPress')}" """                                      + \
                """ onkeydown="${script('keyDown')}"   """                                      + \
                """ onkeyup="${script('keyUp')}"       """                                      + \
                ' >'                                                                            + \
                f'<svg id="mod" width="128" height="128">'                                      + \
                    """\n${mod_inner}\n"""                                                      + \
                '</svg>'                                                                        + \
                '<rect id="drag" x="-10" y="-10" width="5" height="5" stroke="#000000" '        + \
                        'fill="#ffffff" opacity="0.6" />'                                       + \
                f'<rect id="screen" x="0" y="0" width="128" height="128" opacity="0.05"'        + \
                """ onmousedown="${script('downSelect')}"      """                              + \
                """ onmousemove="${script('moveEverything')}"  """                              + \
                """ onmouseup="${script('upEverything')}"      """                              + \
                """ onmousewheel="${script('myonmousewheel')}" """                              + \
                '/>'                                                                            + \
                '<path id="selectionlayer" d="${selectionpath}" fill="#ff0000" transform=""'    + \
                """ onmousedown="${script('downMove')}"       """                               + \
                """ onmousemove="${script('moveEverything')}" """                               + \
                """ onmouseup="${script('upEverything')}"     """                               + \
                '/>'                                                                            + \
                '<rect id="pressrect" x="0"  y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                '<rect id="downrect"  x="10" y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                '<rect id="uprect"    x="20" y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                '</svg>'

    #
    # Constructor
    #
    def __init__(self,
                 rt_self,   # RACETrack instance
                 df,        # data frame
                 ln_params, # linknode params
                 pos,       # position dictionary
                 w,         # Width of the panel
                 h,         # Heght of the panel
                 **kwargs):
        # Setup specific instance information
        # - Copy the member variables
        self.rt_self      = rt_self
        self.ln_params    = ln_params
        self.pos          = pos
        self.w            = w
        self.h            = h
        self.kwargs       = kwargs
        self.df           = self.rt_self.copyDataFrame(df)
        self.df_level     = 0
        self.dfs          = [df]

        # - Create the template ... copy of the above with variables filled in...
        # tabindex="1" 
        self._template = f'<svg tabindex="1" id="myparent" width="{w}" height="{h}" '                       + \
                            """ onkeypress="${script('keyPress')}" """                                      + \
                            """ onkeydown="${script('keyDown')}"   """                                      + \
                            """ onkeyup="${script('keyUp')}"       """                                      + \
                            ' >'                                                                            + \
                            f'<svg id="mod" width="{w}" height="{h}">'                                      + \
                                """\n${mod_inner}\n"""                                                      + \
                            '</svg>'                                                                        + \
                            '<rect id="drag" x="-10" y="-10" width="5" height="5" stroke="#000000" '        + \
                                  'fill="#ffffff" opacity="0.6" />'                                         + \
                            f'<rect id="screen" x="0" y="0" width="{w}" height="{h}" opacity="0.05"'        + \
                            """ onmousedown="${script('downSelect')}"      """                              + \
                            """ onmousemove="${script('moveEverything')}"  """                              + \
                            """ onmouseup="${script('upEverything')}"      """                              + \
                            """ onmousewheel="${script('myonmousewheel')}" """                              + \
                            '/>'                                                                            + \
                            '<path id="selectionlayer" d="${selectionpath}" fill="#ff0000" transform=""'    + \
                            """ onmousedown="${script('downMove')}"       """                               + \
                            """ onmousemove="${script('moveEverything')}" """                               + \
                            """ onmouseup="${script('upEverything')}"     """                               + \
                            '/>'                                                                            + \
                            '<rect id="pressrect" x="0"  y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                            '<rect id="downrect"  x="10" y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                            '<rect id="uprect"    x="20" y="0" width="10" height="10" fill="#a0a0a0" />'    + \
                         '</svg>'
        self.dfs_layout    = [self.__renderView__(self.df)]
        self.mod_inner     = self.dfs_layout[0]._repr_svg_()
        self.selectionpath = "M 0 200 L 100 300 L 0 300 Z"

        # - Create a lock for threading
        self.lock = threading.Lock()

        # Execute the super initialization
        super().__init__(**kwargs)

        # Watch for callbacks
        self.param.watch(self.applyDragOp,   'drag_op_finished')
        self.param.watch(self.applyMoveOp,   'move_op_finished')
        self.param.watch(self.applyWheelOp,  'wheel_op_finished')
        self.param.watch(self.applyMiddleOp, 'middle_op_finished')
    
    #
    # __renderView__() - render the view
    #
    def __renderView__(self, __df__):
        _ln_ = self.rt_self.linkNode(__df__, pos=self.pos, w=self.w, h=self.h, **self.ln_params)
        return _ln_
    
    #
    # Middle button state & method
    #
    x0_middle          = param.Integer(default=0)
    y0_middle          = param.Integer(default=0)
    x1_middle          = param.Integer(default=0)
    y1_middle          = param.Integer(default=0)
    middle_op_finished = param.Boolean(default=False)
    async def applyMiddleOp(self,event):
        self.lock.acquire()
        try:
            if self.middle_op_finished:
                x0, y0, x1, y1 = self.x0_middle, self.y0_middle, self.x1_middle, self.y1_middle
                dx, dy         = x1 - x0, y1 - y0
                _comp_ , _adj_coordinate_ = self.dfs_layout[self.df_level], (x0,y0)
                if _comp_ is not None:
                    if (abs(self.x0_middle - self.x1_middle) <= 1) and (abs(self.y0_middle - self.y1_middle) <= 1):
                        if _comp_.applyMiddleClick(_adj_coordinate_):
                            self.mod_inner     = self.dfs_layout[self.df_level]._repr_svg_() # Re-render current
                            self.selectionpath = self.dfs_layout[self.df_level].__createPathDescriptionOfSelectedEntities__(my_selection=self.selected_entities)                            
                    else:
                        if _comp_.applyMiddleDrag(_adj_coordinate_, (dx,dy)):
                            self.mod_inner     = self.dfs_layout[self.df_level]._repr_svg_() # Re-render current
                            self.selectionpath = self.dfs_layout[self.df_level].__createPathDescriptionOfSelectedEntities__(my_selection=self.selected_entities)
        finally:
            self.middle_op_finished = False
            self.lock.release()

    #
    # Wheel operation state & method
    #
    wheel_x           = param.Integer(default=0)
    wheel_y           = param.Integer(default=0)
    wheel_rots        = param.Integer(default=0) # Mult by 10 and rounded...
    wheel_op_finished = param.Boolean(default=False)
    async def applyWheelOp(self,event):
        self.lock.acquire()
        try:
            if self.wheel_op_finished:
                x, y, rots = self.wheel_x, self.wheel_y, self.wheel_rots
                if rots != 0:
                    # Find the compnent where the scroll event occurred
                    _comp_ , _adj_coordinate_ = self.dfs_layout[self.df_level], (x,y)
                    if _comp_ is not None:
                        if _comp_.applyScrollEvent(rots, _adj_coordinate_):
                            # Re-render current
                            self.mod_inner      = self.dfs_layout[self.df_level]._repr_svg_()
                            self.selectionpath  = self.dfs_layout[self.df_level].__createPathDescriptionOfSelectedEntities__(my_selection=self.selected_entities)                            
                            # Propagate the view configuration to the same component across the dataframe stack
                            for i in range(len(self.dfs_layout)):
                                if i != self.df_level:
                                    self.dfs_layout[i].applyViewConfiguration(_comp_)
        finally:
            self.wheel_op_finished = False
            self.wheel_rots        = 0            
            self.lock.release()

    #
    # Drag operation state & method
    #
    drag_op_finished  = param.Boolean(default=False)
    drag_x0           = param.Integer(default=0)
    drag_y0           = param.Integer(default=0)
    drag_x1           = param.Integer(default=10)
    drag_y1           = param.Integer(default=10)
    last_drag_box     = (0,0,1,1)

    #
    # Move operation state & method
    #
    move_x0          = param.Integer(default=0)
    move_y0          = param.Integer(default=0)
    move_x1          = param.Integer(default=0)
    move_y1          = param.Integer(default=0)
    move_op_finished = param.Boolean(default=False)

    # Key States
    shiftkey         = param.Boolean(default=False)
    ctrlkey          = param.Boolean(default=False)

    #
    # Selected Entities
    #
    selected_entities = []

    #
    # applyDragOp()
    #
    async def applyDragOp(self,event):
        self.lock.acquire()
        try:
            if self.drag_op_finished:
                _x0,_y0,_x1,_y1 = min(self.drag_x0, self.drag_x1), min(self.drag_y0, self.drag_y1), max(self.drag_x1, self.drag_x0), max(self.drag_y1, self.drag_y0)
                if _x0 == _x1: _x1 += 1
                if _y0 == _y1: _y1 += 1
                self.last_drag_box     = (_x0,_y0,_x1-_x0,_y1-_y0)
                _rect_ = Polygon([(_x0,_y0), (_x0,_y1), (_x1,_y1), (_x1,_y0)])
                _overlapping_entities_  = self.dfs_layout[self.df_level].overlappingEntities(_rect_)

                if   self.shiftkey and self.ctrlkey: self.selected_entities = list(set(self.selected_entities) & set(_overlapping_entities_))
                elif self.shiftkey:                  self.selected_entities = list(set(self.selected_entities) - set(_overlapping_entities_))
                elif self.ctrlkey:                   self.selected_entities = list(set(self.selected_entities) | set(_overlapping_entities_))
                else:                                self.selected_entities = _overlapping_entities_
                
                self.selectionpath      = self.dfs_layout[self.df_level].__createPathDescriptionOfSelectedEntities__(my_selection=self.selected_entities)
        finally:
            self.drag_op_finished = False
            self.lock.release()

    async def applyMoveOp(self,event):
        self.lock.acquire()
        try:
            if self.move_op_finished:
                self.dfs_layout[self.df_level].__moveSelectedEntities__((self.move_x1 - self.move_x0, self.move_y1 - self.move_y0), my_selection=self.selected_entities)
                self.mod_inner = self.dfs_layout[self.df_level]._repr_svg_() # Re-render current
                self.move_x0   = self.move_y0 = self.move_x1 = self.move_y1 = 0
                self.selectionpath = self.dfs_layout[self.df_level].__createPathDescriptionOfSelectedEntities__(my_selection=self.selected_entities)
        finally:
            self.move_op_finished = False
            self.lock.release()

    #
    # Panel Javascript Definitions
    #
    _scripts = {
        'render':"""
            mod.innerHTML  = data.mod_inner;
            state.x0_drag  = state.y0_drag = -10;
            state.x1_drag  = state.y1_drag =  -5;
            data.shiftkey = false;
            data.ctrlkey  = false;
            state.drag_op  = false;
            state.move_op  = false;
            data.middle_op_finished = false;
            data.move_op_finished   = false;
        """,
        'keyPress':"""
            pressrect.setAttribute("fill","#ff0000");
        """,
        'keyDown':"""
            downrect.setAttribute("fill","#ff0000");
            if      (event.keyCode == 17) data.ctrlkey  = true;  // keyCode 17
            else if (event.keyCode == 16) data.shiftkey = true;  // keyCode 16
        """,
        'keyUp':"""
            uprect.setAttribute("fill","#ff0000");
            if      (event.keyCode == 17) data.ctrlkey  = false; // keyCode 17
            else if (event.keyCode == 16) data.shiftkey = false; // keyCode 16
        """,
        'moveEverything':"""
            event.preventDefault();
            if (state.drag_op) {
                state.x1_drag  = event.offsetX;
                state.y1_drag  = event.offsetY;
                self.myUpdateDragRect();
            }
            if (state.move_op) {
                state.x1_drag  = event.offsetX;
                state.y1_drag  = event.offsetY;
                selectionlayer.setAttribute("transform", "translate(" + (state.x1_drag - state.x0_drag) + "," + (state.y1_drag - state.y0_drag) + ")");
            }
        """,
        'downSelect':"""
            event.preventDefault();
            if (event.button == 0) {
                state.x0_drag  = event.offsetX;                state.y0_drag  = event.offsetY;                state.x1_drag  = event.offsetX+1;                state.y1_drag  = event.offsetY+1;
                state.drag_op  = true;
                self.myUpdateDragRect();
            } else if (event.button == 1) {
                data.x0_middle = data.x1_middle = event.offsetX;
                data.y0_middle = data.y1_middle = event.offsetY;
            }
        """,
        'downMove':"""
            event.preventDefault();            
            if (event.button == 0) {
                state.x0_drag  = event.offsetX;                state.y0_drag  = event.offsetY;                state.x1_drag  = event.offsetX;                state.y1_drag  = event.offsetY;
                state.move_op  = true;
            } else if (event.button == 1) {
                data.x0_middle = data.x1_middle = event.offsetX;
                data.y0_middle = data.y1_middle = event.offsetY;
            }
        """,
        'upEverything':"""
            event.preventDefault();
            if (state.drag_op && event.button == 0) {
                state.x1_drag  = event.offsetX;                state.y1_drag  = event.offsetY;                state.shiftkey = event.shiftKey;
                state.drag_op  = false;
                self.myUpdateDragRect();
                data.drag_x0          = state.x0_drag;
                data.drag_y0          = state.y0_drag;
                data.drag_x1          = state.x1_drag;
                data.drag_y1          = state.y1_drag;
                data.drag_op_finished = true;
            } else if (event.button == 1) {
                data.x1_middle          = event.offsetX;
                data.y1_middle          = event.offsetY;
                data.middle_op_finished = true;                
            }
            if (state.move_op && event.button == 0) {
                state.x1_drag  = event.offsetX;
                state.y1_drag  = event.offsetY;
                state.move_op  = false;
                data.move_x0   = state.x0_drag;
                data.move_y0   = state.y0_drag;
                data.move_x1   = state.x1_drag;
                data.move_y1   = state.y1_drag;
                data.move_op_finished = true;
            }
        """,
        'myonmousewheel':"""
            event.preventDefault();
            data.wheel_x           = event.offsetX;
            data.wheel_y           = event.offsetY;
            data.wheel_rots        = Math.round(10*event.deltaY);
            data.wheel_op_finished = true;
        """,
        'mod_inner':"""
            mod.innerHTML = data.mod_inner;
        """,
        'selectionpath':"""
            selectionlayer.setAttribute("d", data.selectionpath);
        """,
        'myUpdateDragRect':"""
            if (state.drag_op) {
                x = state.x0_drag; 
                if (state.x1_drag < x) { x = state.x1_drag; }
                y = state.y0_drag; 
                if (state.y1_drag < y) { y = state.y1_drag; }
                w = Math.abs(state.x1_drag - state.x0_drag)
                h = Math.abs(state.y1_drag - state.y0_drag)
                drag.setAttribute('x',x);     drag.setAttribute('y',y);
                drag.setAttribute('width',w); drag.setAttribute('height',h);
                if      (data.shiftkey && data.ctrlkey) drag.setAttribute('stroke','#0000ff');
                else if (data.shiftkey)                 drag.setAttribute('stroke','#ff0000');
                else if                  (data.ctrlkey) drag.setAttribute('stroke','#00ff00');
                else                                    drag.setAttribute('stroke','#000000');
            } else {
                drag.setAttribute('x',-10);   drag.setAttribute('y',-10);
                drag.setAttribute('width',5); drag.setAttribute('height',5);
            }
        """
    }

df = pd.DataFrame({'fm':'a b c d e'.split(),
                   'to':'b c d e a'.split()})
_params_ = {'relationships':[('fm','to')], 'draw_labels':True}
_pos_    = {'a':(0,0), 'b':(1,0), 'c':(1,1), 'd':(0.5,1), 'e':(0,1)}
_rtg_ = RTGraphInteractHTML(rt, df, _params_, _pos_, 400, 400)
_rtg_

In [ ]:
_rtg_.last_drag_box, _rtg_.selected_entities, _rtg_.selectionpath

In [ ]:
rt.linkNode(df, pos=_pos_, **_params_)

In [ ]:
#df_nf  = pl.read_csv('../../netflow_sample.csv').sample(100_000)
#print(len(set(df_nf['sip']) | set(df_nf['dip'])))
#pos_nf = {}

In [ ]:
#_rtg_nf_ = RTGraphInteractHTML(rt, df_nf, {'relationships':[('sip','dip')], 'draw_labels':False, 'link_opacity':0.2}, pos_nf, 400, 400)
#_rtg_nf_

In [ ]:
# ... that leaves "cehimstuyz" as the only safe keys for shift & control combinations
# ... 1..9 are all safe... shift + 1..9 are all safe... but not ctrl + 1..9
# ... usually - "shift" is subtract, "ctrl" is add, and "shift-ctrl" is intersect

## ICONS (along bottom, left to right)
# reset view (all nodes)
# reset view (selected nodes)
# toggle labels / toggle sticky labels / toggle no labels
# tagging? / done the right hand side of the view?

## LAYOUT
# 'c': grid (most common use)
# 'c': shift - circle
# 'c': ctrl  - circle - sunflower
# 't': line - w/ restrictions for (shift) horizontal and/or (ctrl) vertical lines
# 't': collapse to a single point

## SHORTCUTS
# 'e'     : expand selection ... shift is 'select only in direction of edge'
# 'e'     : 'ctrl' is invert selection, 'shift-ctrl' is select common neighbors
# 'z'     : select color under the mouse / shift-subtract / ctrl-add
# '1..90' : select node by degree 1...9 ... 0 is degree 10 or more // only shift is safe... can't use control
# 's'     : add / remove from sticky labels
# 'spc'   : remove selected nodes from the graph , 'ctrl-spc' to add nodes back in (go up the stack)

## INTERACTION
# select / add to select / subtract from select / intersect with select
